## Bootstrap Formatted Popup

Example code that shows how to include Bootstrap in Pop up.
This example is simply to add a button - should be possible to add a carousel the same way. Note that HTML has to be previewed in browser as iFrame won't display here.

### Initial imports and create df from CSV


In [7]:
import pandas as pd
import folium
import re

df = pd.read_csv("20250304_data.csv", encoding='utf-8')


### Convert DMS Coordinates to Decimal



In [8]:
def dms_to_decimal(dms_str):
    match = re.match(r"(\d+)°(\d+)'(\d+)\"([NSEW])", dms_str)
    if not match:
        return None  # Handle bad data gracefully

    degrees, minutes, seconds, direction = match.groups()
    decimal = float(degrees) + float(minutes) / 60 + float(seconds) / 3600

    # Make it negative for South or West
    if direction in "SW":
        decimal = -decimal

    return round(decimal, 6)  # Round for accuracy

# Apply function to the DataFrame
df["lat_dec"] = df["lat"].apply(dms_to_decimal)
df["long_dec"] = df["long"].apply(dms_to_decimal)

# Display results
df.head()

,name,date,OS,lat,long,altitude,lat_dec,long_dec,Image_01,Image_02
0,Ruin past the White House on the Rubha Dunan p...,23/04/2024,NC 02440 07127,"58°0'33""N","5°20'44""W",19,58.009167,-5.345556,https://github.com/MorningGlass/coigach-herita...,https://github.com/MorningGlass/coigach-herita...


### Create map

In [9]:
# Define map center
map_center = [df["lat_dec"].mean(), df["long_dec"].mean()]
m = folium.Map(location=map_center, zoom_start=11, tiles="OpenStreetMap", name="OpenStreetMap")

# Add Google Satellite Layer
folium.TileLayer(
    tiles="https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}",
    attr="Google Satellite",
    name="Google Satellite",
    overlay=False,
    control=True,
    show=False
).add_to(m)

# Add Layer Control
folium.LayerControl().add_to(m)


# Add entry for each ruin
for i, row in df.iterrows():
    # Unique carousel ID for each popup to avoid conflicts
    carousel_id = f"carouselExample{i}"

    # HTML & CSS for Bootstrap-styled popup inside an IFrame
    popup_html = f"""
    <!DOCTYPE html>
    <html lang="en">
    <head>
        <meta name="viewport" content="width=device-width, initial-scale=0.5">
        <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.3.3/dist/css/bootstrap.min.css" rel="stylesheet">
    </head>
    <body>
        <div class="container p-2 my-3">
            <h4 class="text-dark">{row['name']}</h4>
            <p><b>OS Grid Ref:</b> {row['OS']}</p>
            <p><b>Location: </b>{row['lat']}, {row['long']}</p>
            <p><b>Date photographed:</b> {row['date']}</p>
            <p><b>Altitude:</b> {row['altitude']}m</p>

            <div id="{carousel_id}" class="carousel slide" data-bs-ride="true">
              <div class="carousel-inner">
                <div class="carousel-item active">
                  <img src="{row['Image_01']}" alt="Image of {row['name']}" class="d-block w-100">
                </div>
                <div class="carousel-item">
                  <img src="{row['Image_02']}" alt="Image of {row['name']}" class="d-block w-100">
                </div>
              </div>
              <button class="carousel-control-prev" type="button" data-bs-target="#{carousel_id}" data-bs-slide="prev">
                <span class="carousel-control-prev-icon" aria-hidden="true"></span>
                <span class="visually-hidden">Previous</span>
              </button>
              <button class="carousel-control-next" type="button" data-bs-target="#{carousel_id}" data-bs-slide="next">
                <span class="carousel-control-next-icon" aria-hidden="true"></span>
                <span class="visually-hidden">Next</span>
              </button>
            </div>

            <div class="text-center mt-2">
                <a href="contact.html" class="btn btn-outline-secondary my-3">Submit Info about this ruin</a>
            </div>
        </div>

        <!-- Load Bootstrap JavaScript (Required for Carousel) -->
        <script src="https://cdn.jsdelivr.net/npm/bootstrap@5.3.3/dist/js/bootstrap.bundle.min.js"></script>
    </body>
    </html>
    """

    # Use an IFrame to ensure Bootstrap works inside the popup
    iframe = folium.IFrame(html=popup_html, width=400, height=600)
    popup = folium.Popup(iframe, max_width=400)

    # Add marker with popup
    folium.Marker(
        location=[row["lat_dec"], row["long_dec"]],
        popup=popup
    ).add_to(m)

# Save and display map
m.save("map_with_bootstrap.html")
m
